In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  

</style>

In [2]:
# imports
import datetime
import pprint
import pyaurorax
aurorax = pyaurorax.PyAuroraX()


#  To begin, let's do an ephemeris search for some THEMIS ASI data.

# We will search for any data between these dates
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
programs = ["themis-asi"]

# Now, to filter based on ML data, we need to set up some metadata
# filters. Let's filter to only retrieve records which the
# 'calgary_apa_ml_v1' ML dataset classified as APA, with a confidence
# greater than or equal to 95%.
#
# This is done as below ...
metadata_filters_logical_operator = "AND"
metadata_filters = [
          {
            "key": "calgary_apa_ml_v1",     #
            "operator": "in",               #   Here, we set up the metadata filter to retrieve
            "values": [                     #   only records that were classified as APA....
              "classified as APA"           #
            ]
          },
          {
            "key": "calgary_apa_ml_v1_confidence",
            "operator": ">=",                         # ... with a confidence of 
            "values": [                               # at least 95%
              "95"
            ]
          }
        ]

# Perform the search
s = aurorax.search.ephemeris.search(
    start=start,
    end=end,
    programs=programs,
    metadata_filters_logical_operator=metadata_filters_logical_operator,
    metadata_filters=metadata_filters,
    verbose=True)

[2024-11-27 14:21:46.148471] Search object created
[2024-11-27 14:21:46.180106] Request submitted
[2024-11-27 14:21:46.180126] Request ID: 88123fa8-f005-40d9-892a-542151629bf7
[2024-11-27 14:21:46.180130] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/88123fa8-f005-40d9-892a-542151629bf7
[2024-11-27 14:21:46.180133] Waiting for data ...
[2024-11-27 14:21:47.620988] Checking for data ...
[2024-11-27 14:21:48.064039] Data is now available
[2024-11-27 14:21:48.064265] Retrieving data ...
[2024-11-27 14:21:49.915099] Retrieved 18.7 MB of data containing 5407 records


In [3]:
# Print the first 10 records
pprint.pprint(s.data[0:10])

[EphemerisData(epoch=datetime.datetime(2008, 1, 1, 11, 30), location_geo=Location(lat=61.755798, lon=-121.227005), location_gsm=Location(lat=None, lon=None), nbtrace=Location(lat=61.755798, lon=-121.227005), sbtrace=Location(lat=-64.7499007670771, lon=-172.0643415195183), metadata={'clausen_ml_oath': ...}, data_source=DataSource(...)),
 EphemerisData(epoch=datetime.datetime(2008, 1, 1, 11, 31), location_geo=Location(lat=61.755798, lon=-121.227005), location_gsm=Location(lat=None, lon=None), nbtrace=Location(lat=61.755798, lon=-121.227005), sbtrace=Location(lat=-64.74990082859775, lon=-172.06434121158588), metadata={'clausen_ml_oath': ...}, data_source=DataSource(...)),
 EphemerisData(epoch=datetime.datetime(2008, 1, 1, 11, 32), location_geo=Location(lat=61.755798, lon=-121.227005), location_gsm=Location(lat=None, lon=None), nbtrace=Location(lat=61.755798, lon=-121.227005), sbtrace=Location(lat=-64.74990089011834, lon=-172.06434090365335), metadata={'clausen_ml_oath': ...}, data_source=

In [4]:
# Access the classification of interest
print("Classification:", s.data[0].metadata['calgary_apa_ml_v1'])
print("Confidence:", s.data[0].metadata['calgary_apa_ml_v1_confidence'])

Classification: classified as APA
Confidence: 99.96


In [5]:
# Let's print the results of the first 10 records as a table
print(f"{'Timestamp':<{25}}"
      f"{'calgary_apa_ml_v1':<{20}} "
      f"{'confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_apa_ml_v1']:<{20}} "
          f"  {s.data[i].metadata['calgary_apa_ml_v1_confidence']:<{18}} ")

Timestamp                calgary_apa_ml_v1    confidence        
2008-01-01 11:30:00      classified as APA      99.96              
2008-01-01 11:31:00      classified as APA      99.96              
2008-01-01 11:32:00      classified as APA      99.96              
2008-01-01 11:33:00      classified as APA      99.96              
2008-01-01 11:34:00      classified as APA      99.96              
2008-01-01 11:35:00      classified as APA      99.96              
2008-01-01 11:36:00      classified as APA      99.96              
2008-01-01 11:37:00      classified as APA      99.96              
2008-01-01 11:38:00      classified as APA      99.96              
2008-01-01 11:39:00      classified as APA      99.96              


In [6]:
# Now, let's instead search for data, and filter out cloudy
# data. We set up our metadata filter as follows, to filter
# and only retrieve data that was classified as not cloudy
# with a confidence > 90%

# set up search parameters, we will restrict this search to one site, Gillam
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 7, 23, 59, 59)
programs = ["themis-asi"]
platforms = ["gillam"]

metadata_filters_logical_operator = "AND"
metadata_filters = [
          {
            "key": "calgary_cloud_ml_v1",
            "operator": "in",
            "values": [
              "classified as not cloudy"
            ]
          },
          {
            "key": "calgary_cloud_ml_v1_confidence",
            "operator": ">",
            "values": [
              "90"
            ]
          }
        ]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    platforms=platforms,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2024-11-27 14:21:49.937189] Search object created
[2024-11-27 14:21:49.956997] Request submitted
[2024-11-27 14:21:49.957020] Request ID: 597609e8-0a61-470d-a879-7677d70df75a
[2024-11-27 14:21:49.957024] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/597609e8-0a61-470d-a879-7677d70df75a
[2024-11-27 14:21:49.957027] Waiting for data ...
[2024-11-27 14:21:51.374502] Checking for data ...
[2024-11-27 14:21:51.795770] Data is now available
[2024-11-27 14:21:51.796000] Retrieving data ...
[2024-11-27 14:21:52.089026] Retrieved 2.0 MB of data containing 565 records


In [7]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{28}} "
      f"{'Confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_cloud_ml_v1']:<{28}} "
          f"  {s.data[i].metadata['calgary_cloud_ml_v1_confidence']:<{18}} ")

Timestamp                Classification               Confidence        
2008-01-01 23:28:00      classified as not cloudy       91.81              
2008-01-01 23:29:00      classified as not cloudy       91.81              
2008-01-02 00:00:00      classified as not cloudy       99.96              
2008-01-02 00:01:00      classified as not cloudy       99.96              
2008-01-02 00:02:00      classified as not cloudy       99.96              
2008-01-02 00:03:00      classified as not cloudy       99.96              
2008-01-02 00:04:00      classified as not cloudy       99.96              
2008-01-02 00:05:00      classified as not cloudy       99.96              
2008-01-02 00:06:00      classified as not cloudy       99.96              
2008-01-02 00:07:00      classified as not cloudy       99.96              


In [8]:
# Finally, let's gather some data based on classifications
# using the OATH model. Let's filter so that we obtain
# records that were classified as either discrete or diffuse aurora.

# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 7, 23, 59, 59)
programs = ["themis-asi"]

metadata_filters_logical_operator = "AND"
metadata_filters = [
          {
            "key": "clausen_ml_oath",
            "operator": "in",
            "values": [
              "classified as diffuse",
              "classified as discrete",
            ]
          }
        ]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2024-11-27 14:21:52.102043] Search object created
[2024-11-27 14:21:52.123472] Request submitted
[2024-11-27 14:21:52.123495] Request ID: 2e5a9be2-24d5-4a57-9cf5-5081c440dc56
[2024-11-27 14:21:52.123498] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/2e5a9be2-24d5-4a57-9cf5-5081c440dc56
[2024-11-27 14:21:52.123501] Waiting for data ...
[2024-11-27 14:21:53.546379] Checking for data ...
[2024-11-27 14:21:54.969656] Checking for data ...
[2024-11-27 14:21:55.382282] Data is now available
[2024-11-27 14:21:55.382520] Retrieving data ...
[2024-11-27 14:21:58.515132] Retrieved 31.6 MB of data containing 9156 records


In [9]:
# Again, print some results
print(f"{'Timestamp':<{28}}"
      f"{'Classification':<{18}}")
print("=================================================")

for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{23}}"
          f"  {s.data[i].metadata['clausen_ml_oath']:<{18}} ")

Timestamp                   Classification    
2008-01-01 01:11:00      classified as discrete 
2008-01-01 01:19:00      classified as diffuse 
2008-01-01 01:20:00      classified as diffuse 
2008-01-01 01:22:00      classified as discrete 
2008-01-01 01:25:00      classified as diffuse 
2008-01-01 01:26:00      classified as diffuse 
2008-01-01 01:27:00      classified as diffuse 
2008-01-01 01:28:00      classified as diffuse 
2008-01-01 01:29:00      classified as diffuse 
2008-01-01 01:30:00      classified as diffuse 
